In [69]:
import os
import pandas as pd
import sys
PROJECT_ROOT = os.path.abspath("..")
sys.path.insert(0, PROJECT_ROOT)
from script.csvs_to_sql import detect_delimiter
CSV_PATH = "../data_acidentes_prf/"
year = "2017"


In [70]:
def files_per_year(year:str|int) -> list[str]:
    year = str(year)
    files_per_year = []
    files = os.listdir(CSV_PATH)
    for file in files:
        if year in file:
            files_per_year.append(file)
    return files_per_year

def concat_df(files:list[str]) -> pd.DataFrame:
    dfs = [
    pd.read_csv(
        f"{CSV_PATH}{file}",
        encoding="latin1",
        sep=detect_delimiter(f"{CSV_PATH}{file}")
    )
    for file in files
    ]

    df = pd.concat(dfs, ignore_index=True)
    df = df.drop_duplicates()
    return df

files = files_per_year(year)
df = concat_df(files)
df.shape


/tmp/ipykernel_70742/3176775890.py:12: DtypeWarning: Columns (22,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(


(656301, 41)

In [71]:
df.columns


Index(['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br',
       'km', 'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo',
       'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos',
       'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude',
       'regional', 'delegacia', 'uop', 'pessoas', 'ignorados', 'feridos',
       'veiculos', 'causa_principal', 'ordem_tipo_acidente'],
      dtype='object')

### Exibindo os 10 municípios que tiveram mais mortes no ano correspondente

In [72]:
try:
  Top_10_most_dangerous_municipios = (
      df.groupby(["municipio", "uf"], as_index=False)["mortos"]
        .sum()
        .nlargest(10, "mortos")
        .reset_index(drop=True)
  )
except  KeyError as e:
  print("The selected columns do not exits")

Top_10_most_dangerous_municipios


,municipio,uf,mortos
0,GUARAPARI,ES,640.0
1,CURITIBA,PR,217.0
2,MIMOSO DO SUL,ES,198.0
3,BRASILIA,DF,167.0
4,CAMPO GRANDE,MS,132.0
5,VITORIA DA CONQUISTA,BA,126.0
6,LUZIANIA,GO,123.0
7,BETIM,MG,113.0
8,GUARULHOS,SP,108.0
9,TERESINA,PI,102.0


### Dias da semana e horários com mais acidentes

In [73]:
df_seriously_injured = df.query("feridos_graves > 0")

try:
    days_of_the_week_with_the_most_accidents = (
        df_seriously_injured.groupby(["dia_semana"], as_index=False)
        .size()
        .rename(columns={"size": "qtd_acidentes"})
        .sort_values("qtd_acidentes", ascending=False)
        .nlargest(10, "qtd_acidentes")
    )
except KeyError as e:
    print("The selected columns do not exits")

days_of_the_week_with_the_most_accidents


,dia_semana,qtd_acidentes
0,domingo,12489
5,sábado,11478
4,sexta-feira,9202
3,segunda-feira,7991
2,quinta-feira,7397
1,quarta-feira,7099
6,terça-feira,7032


### Horários com mais acidentes

In [74]:
try:
    time_with_the_most_serious_accidents = (
        df_seriously_injured.groupby(["horario"], as_index=False)
        .size()
        .rename(columns={"size": "qtd_acidentes"})
        .sort_values("qtd_acidentes", ascending=False)
        .nlargest(10, "qtd_acidentes")
    )
except KeyError as e:
    print("The selected columns do not exits")
time_with_the_most_serious_accidents

,horario,qtd_acidentes
606,19:00:00,1133
561,18:00:00,1031
582,18:30:00,945
649,20:00:00,725
628,19:30:00,721
543,17:30:00,705
480,16:00:00,698
521,17:00:00,659
591,18:40:00,602
499,16:30:00,592


### As principais causas de acidentes que tiveram mais mortos

In [ ]:
main_cause = (
    df.query("mortos > 0")
      .groupby("causa_acidente", as_index=False)
      .size()
      .rename(columns={"size": "qtd_acidentes_com_mortes"})
      .sort_values("qtd_acidentes_com_mortes", ascending=False)
)
main_cause

,causa_acidente,qtd_acidentes_com_mortes
9,Falta de Atenção à Condução,6400
20,Velocidade Incompatível,4016
8,Falta de Atenção do Pedestre,2196
12,Ingestão de Álcool,1598
7,Desobediência às normas de trânsito pelo condutor,1470
19,Ultrapassagem Indevida,1391
3,Condutor Dormindo,1274
16,Pista Escorregadia,633
13,Mal Súbito,616
4,Defeito Mecânico no Veículo,570
